In [1]:
import sys
import os.path
import pylab as plt
from zedstat import zedstat 
import pandas as pd
plt.style.use('fivethirtyeight')

In [2]:
#
# order is order of spline used for smoothing
# alpha is significance level
# prevalence is prevalence of positive class
# 
zt=zedstat.processRoc(df=pd.read_csv('examples_data/roc2.csv'),
           order=3, 
           total_samples=100000,
           positive_samples=100,
           alpha=0.01,
           prevalence=.002)

NameError: name 'random_state' is not defined

In [ ]:
# compute smoothened roc and other performance measures
zt.smooth(STEP=0.001)
zt.allmeasures(interpolate=True)
zt.usample(precision=3)
zt.getBounds()

In [ ]:
ax=plt.gca()
ax=zt.get().tpr.plot(ax=ax,color='k')
plt.fill_between(x=zt.get().index, y1=zt.df_lim['U'].tpr,y2=zt.df_lim['L'].tpr, color='k',alpha=.2)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_ylabel('tpr');

In [ ]:
ax=plt.gca()
ax=zt.get().plot(ax=ax,x='tpr',y='ppv',color='k')
plt.fill_between(x=zt.get()['tpr'], y1=zt.df_lim['U'].ppv,y2=zt.df_lim['L'].ppv, color='k',alpha=.2)
ax.set_xlim(0,1)
ax.set_ylim(0,1)

In [ ]:
# compute auc with confidence bounds
zt.auc(alpha=.05)

In [ ]:
# required sample size
zt.samplesize(delta_auc=.05)

In [ ]:
import numpy as np
pd.DataFrame(np.array([zt.samplesize(delta_auc=a) 
                       for a in np.arange(.05,.1,.001)]),
             np.arange(.05,.1,.001)).plot(logy=True)
plt.gca().set_xlabel('delta AUC')
plt.gca().set_ylabel('minimum samples');

In [ ]:
pd.DataFrame([zt.pvalue(delta_auc=a) for a in np.arange(0.001,.1,.001)],
             np.arange(0.001,.1,.001)).plot()
plt.gca().set_ylim(0,0.05)   
#plt.gca().set_xlim(0.0196,0.035)   
plt.gca().set_xlabel('delta AUC')
plt.gca().set_ylabel('pvalue');

In [ ]:
zt.operating_zone(LRminus=.65)
rf0,txt0=zt.interpret(fpr=zt._operating_zone.fpr.values[0],number_of_positives=10)
rf1,txt1=zt.interpret(fpr=zt._operating_zone.fpr.values[1],number_of_positives=10)
display(zt._operating_zone)
print('high precision operation:\n','\n '.join(txt0))
print('\n')
print('high recall operation:\n','\n '.join(txt1))

In [ ]:
#pd.options.display.html.use_mathjax = False
from zedstat.textable import tablewithbounds
xf=tablewithbounds(zt.get(),zt.usample(zt.df_lim['U']),zt.usample(zt.df_lim['L']))#.to_csv('table.csv')
xf.loc[zt._operating_zone.fpr.values]

In [ ]:
from zedstat.textable import textable
textable(xf,tabname='tmp.tex',FORMAT='%1.3f',INDEX=True,LNTERM='\\\\\\hline\n')     

# Second example: From raw samples

In [ ]:
pf=pd.read_csv('./examples_data/id_predictions_M_2.csv',index_col=0)
pf

In [ ]:
rf,total_samples,positive_samples=zedstat.genroc(pf,risk='predicted_risk',
           target='target',
           TARGET=[1])

In [ ]:
rf.plot(x='fpr',y='tpr')

In [ ]:
zt1=zedstat.processRoc(df=rf,
           order=3, 
           total_samples=total_samples,
           positive_samples=positive_samples,
           alpha=0.05,
           prevalence=.002)

In [ ]:
zt1.smooth(STEP=0.001)
zt1.allmeasures(interpolate=True)
zt1.usample(precision=3)
zt1.getBounds()
ax=plt.gca()
ax=zt1.get().tpr.plot(ax=ax,color='k')
plt.fill_between(x=zt1.get().index, y1=zt1.df_lim['U'].tpr,y2=zt1.df_lim['L'].tpr, color='k',alpha=.2)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_ylabel('tpr');
print(zt1.auc(alpha=.05))

In [ ]:
zt1.samplesize(delta_auc=.1)